## 1. Install Dependencies

In [1]:
!pip install -q transformers>=4.46.0 "datasets>=3.0.0,<4.0.0" trl>=0.12.0 peft>=0.13.0 accelerate>=1.0.0 bitsandbytes>=0.44.0 scikit-learn pandas

## 1.1 HuggingFace Token (optional, set once)


In [ ]:
# HuggingFace credentials (set once at the top)
HF_TOKEN = ""  # Paste your token here
HF_USERNAME = "ehzawad"  # Change if needed
HF_REPO_SUFFIX = "bn-nid-intent-qwen2.5-0.5b"  # Contextual repo name
HF_RUN_TAG = ""  # Optional: set a custom tag (auto-filled if empty)


## 2. Check GPU & Mount Google Drive

In [2]:
import torch

# Check GPU
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {gpu_memory:.1f} GB")
else:
    print("No GPU available! Go to Runtime > Change runtime type > GPU")
    raise RuntimeError("GPU required")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create output directories for model/checkpoints
import os

RUN_NAME = "smollm2-bengali-nid-intent"  # Stable name for reloads
OUTPUT_DIR = f"/content/drive/MyDrive/models/{RUN_NAME}"
DPO_OUTPUT_DIR = f"{OUTPUT_DIR}-dpo"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(DPO_OUTPUT_DIR, exist_ok=True)
print(f"Model output directory: {OUTPUT_DIR}")
print(f"DPO output directory: {DPO_OUTPUT_DIR}")

# ============================================================
# DATASET PATH - Upload your CSVs to this folder in Google Drive
# ============================================================
DATASET_DIR = "/content/drive/MyDrive"
os.makedirs(DATASET_DIR, exist_ok=True)

print(f"\n>>> Upload your CSV files to: {DATASET_DIR}")
print("    - sts_train.csv")
print("    - sts_eval.csv")
print("    - tag_answer.csv")
print("\nOr change DATASET_DIR to where your files are located.")


GPU: NVIDIA L4
VRAM: 23.8 GB
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model output directory: /content/drive/MyDrive/models/smollm2-bengali-nid-intent

>>> Upload your CSV files to: /content/drive/MyDrive
    - sts_train.csv
    - sts_eval.csv
    - tag_answer.csv

Or change DATASET_DIR to where your files are located.


## 3. Verify Dataset Files

Make sure your CSV files are in Google Drive at the path shown above.

In [3]:
import os

# Check if files exist in Google Drive
train_path = f"{DATASET_DIR}/sts_train.csv"
eval_path = f"{DATASET_DIR}/sts_eval.csv"
tag_path = f"{DATASET_DIR}/tag_answer.csv"

files_status = {
    "sts_train.csv": os.path.exists(train_path),
    "sts_eval.csv": os.path.exists(eval_path),
    "tag_answer.csv": os.path.exists(tag_path),
}

print("Dataset files status:")
for fname, exists in files_status.items():
    status = "✓ Found" if exists else "✗ Missing"
    print(f"  {status}: {fname}")

if not all(files_status.values()):
    missing = [f for f, exists in files_status.items() if not exists]
    print(f"\n❌ Missing files: {missing}")
    print(f"Please upload them to: {DATASET_DIR}")
    raise FileNotFoundError(f"Missing dataset files in {DATASET_DIR}")
else:
    print(f"\n✓ All files found in {DATASET_DIR}")

Dataset files status:
  ✓ Found: sts_train.csv
  ✓ Found: sts_eval.csv
  ✓ Found: tag_answer.csv

✓ All files found in /content/drive/MyDrive


## 4. Load and Analyze Dataset

In [4]:
import pandas as pd
from collections import Counter

# Load CSV files from Google Drive
print("Loading dataset files from Google Drive...")
train_df = pd.read_csv(train_path)
# ============================================================
# SAMPLE 50% OF TRAINING DATA FOR FASTER TRAINING
# ============================================================
train_df = train_df.sample(frac=0.5, random_state=42).reset_index(drop=True)
print(">>> Using 50% of training data for faster training <<<")
eval_df = pd.read_csv(eval_path)
tag_answer_df = pd.read_csv(tag_path)

print(f"Train samples: {len(train_df)}")
print(f"Eval samples: {len(eval_df)}")
print(f"Unique tags in train: {train_df['tag'].nunique()}")
print(f"Unique tags in eval: {eval_df['tag'].nunique()}")
print(f"Tags with answers: {len(tag_answer_df)}")

# Show sample
print(f"\nSample from training data:")
print(f"  Question: {train_df.iloc[0]['question']}")
print(f"  Tag: {train_df.iloc[0]['tag']}")

Loading dataset files from Google Drive...
Train samples: 78616
Eval samples: 11457
Unique tags in train: 407
Unique tags in eval: 403
Tags with answers: 407

Sample from training data:
  Question: "একাউন্ট লক করা হয়েছে" দেখাচ্ছে, সমাধান কী?
  Tag: account_locked


In [5]:
# Build intent labels from training data
INTENT_TAGS = sorted(train_df['tag'].unique().tolist())
print(f"Total unique intents: {len(INTENT_TAGS)}")

# Create mappings
ID2INTENT = {i: intent for i, intent in enumerate(INTENT_TAGS)}
INTENT2ID = {intent: i for i, intent in enumerate(INTENT_TAGS)}

# Show top 15 tags by frequency
print(f"\nTop 15 tags by frequency:")
tag_counts = train_df['tag'].value_counts()
for tag, count in tag_counts.head(15).items():
    print(f"  {tag}: {count}")

Total unique intents: 407

Top 15 tags by frequency:
  fraction: 494
  permanent_address_change_fees: 381
  spouse_name_correction_new: 231
  parent_spouse_name_correct_or_add_document_new: 229
  parents_name_correction_new: 226
  goodbye: 218
  picture_done_but_lost_or_no_sms_slip: 215
  service_provided: 213
  disability_no_hands_registration_procedure: 206
  abroad_smart_card_collection_return: 206
  reissue_urgent_card_delivery_time: 206
  signature_to_fingerprint_reversal_not_allowed: 206
  reissue_smart_card_download_not_available: 206
  abroad_illegal_resident_nid: 206
  abroad_embassy_walk_in_registration: 206


## 5. Configuration

In [ ]:
# ============================================================
# CONFIGURATION - Direct Classification (No CoT, No Few-Shot)
# ============================================================

# Model - Qwen2.5-0.5B-Instruct for multilingual support
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"

# Data - Short sequences since output is just a tag
MAX_SEQ_LENGTH = 256  # Minimal: query + tag only

# Training - Optimized for L4 24GB (~18GB usage)
NUM_EPOCHS = 3
BATCH_SIZE = 16       # Large batch for short sequences
EVAL_BATCH_SIZE = 32  # Even larger for eval (no gradients)
GRAD_ACCUM_STEPS = 4  # Effective batch = 64
LEARNING_RATE = 2e-4  # Higher LR for direct classification
WARMUP_RATIO = 0.05
EARLY_STOPPING_PATIENCE = 3

# LoRA config
LORA_R = 32
LORA_ALPHA = 64
LORA_DROPOUT = 0.05
LORA_TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

# Seed
SEED = 42

# Checkpoint resume (optional)
RESUME_FROM_CHECKPOINT = None  # e.g., f"{OUTPUT_DIR}/checkpoint-1500"
DPO_RESUME_FROM_CHECKPOINT = None  # e.g., f"{DPO_OUTPUT_DIR}/checkpoint-100"

print(f"Model: {MODEL_NAME}")
print(f"Number of intents: {len(INTENT_TAGS)}")
print(f"Max sequence length: {MAX_SEQ_LENGTH}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Batch size: {BATCH_SIZE} x {GRAD_ACCUM_STEPS} = {BATCH_SIZE * GRAD_ACCUM_STEPS} effective")
print(f"Learning rate: {LEARNING_RATE}")
print(f"LoRA rank: {LORA_R}")


## 6. Prepare Dataset for SFT

In [ ]:
from datasets import Dataset
from tqdm import tqdm
import random

# ============================================================
# SIMPLIFIED FORMAT: Direct Classification
# System: You are an intent classifier. Output only the intent tag.
# User: {query}
# Assistant: {tag}
# ============================================================

# System prompt - minimal and direct
SYSTEM_PROMPT = "You are an intent classifier for Bengali NID customer service. Output only the intent tag, nothing else."

def format_for_direct_classification(row):
    """Create minimal chat format for direct intent classification."""
    question = row['question']
    target_tag = row['tag']
    
    return {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": question},
            {"role": "assistant", "content": target_tag}
        ],
        "intent": target_tag
    }

# ============================================================
# Build Training and Eval Datasets
# ============================================================

print("Formatting training data (direct classification)...")
random.seed(SEED)
train_formatted = []
for _, row in tqdm(train_df.iterrows(), total=len(train_df), desc="Train"):
    formatted = format_for_direct_classification(row)
    train_formatted.append(formatted)

train_dataset = Dataset.from_list(train_formatted)

print("Formatting evaluation data...")
eval_formatted = []
for _, row in tqdm(eval_df.iterrows(), total=len(eval_df), desc="Eval"):
    formatted = format_for_direct_classification(row)
    eval_formatted.append(formatted)

eval_dataset = Dataset.from_list(eval_formatted)

print(f"\nTrain dataset: {len(train_dataset)} samples")
print(f"Eval dataset: {len(eval_dataset)} samples")

# ============================================================
# DEBUG: Token length statistics
# ============================================================
print(f"\n{'='*60}")
print("TOKEN LENGTH DEBUG (sampling 100 examples)")
print(f"{'='*60}")

sample_indices = random.sample(range(len(train_formatted)), min(100, len(train_formatted)))
token_lengths = []
for idx in sample_indices:
    sample = train_formatted[idx]
    # Concatenate all message contents
    full_text = ''.join([m['content'] for m in sample['messages']])
    tokens = len(full_text.split())  # Rough word count
    token_lengths.append(tokens)

print(f"  Min word count: {min(token_lengths)}")
print(f"  Max word count: {max(token_lengths)}")
print(f"  Mean word count: {sum(token_lengths)/len(token_lengths):.1f}")
print(f"  Max seq length setting: {MAX_SEQ_LENGTH}")

# Show formatted sample
print(f"\n{'='*60}")
print("SAMPLE FORMATTED TRAINING EXAMPLE:")
print(f"{'='*60}")
sample = train_dataset[0]
for msg in sample['messages']:
    print(f"\n[{msg['role'].upper()}]")
    print(msg['content'])

# Show a few more examples
print(f"\n{'='*60}")
print("MORE EXAMPLES (showing input -> output):")
print(f"{'='*60}")
for i in range(min(5, len(train_dataset))):
    sample = train_dataset[i]
    user_msg = sample['messages'][1]['content']
    assistant_msg = sample['messages'][2]['content']
    print(f"  Q: {user_msg[:60]}...")
    print(f"  A: {assistant_msg}")
    print()

## 7. Load Model and Apply LoRA

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, TaskType, get_peft_model

# Load tokenizer
print(f"Loading tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# IMPORTANT: Use left-padding for decoder-only models during batched generation
tokenizer.padding_side = 'left'

# Debug: Tokenizer info
print(f"\n{'='*60}")
print("TOKENIZER DEBUG INFO")
print(f"{'='*60}")
print(f"  Vocab size: {tokenizer.vocab_size}")
print(f"  Pad token: {tokenizer.pad_token} (id={tokenizer.pad_token_id})")
print(f"  EOS token: {tokenizer.eos_token} (id={tokenizer.eos_token_id})")
print(f"  Padding side: {tokenizer.padding_side}")
print(f"  Has chat template: {tokenizer.chat_template is not None}")

# Test chat template
test_msgs = [{"role": "user", "content": "test"}]
test_prompt = tokenizer.apply_chat_template(test_msgs, tokenize=False, add_generation_prompt=True)
print(f"  Chat template test: '{test_prompt[:100]}...'")

# Load model
print(f"\n{'='*60}")
print(f"LOADING MODEL: {MODEL_NAME}")
print(f"{'='*60}")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)
model.to(device)

# Disable cache and enable gradient checkpointing for training
model.config.use_cache = False
model.gradient_checkpointing_enable()
if hasattr(model, "enable_input_require_grads"):
    model.enable_input_require_grads()

# Debug: Model info
print(f"\nMODEL DEBUG INFO")
print(f"  Total parameters: {model.num_parameters():,}")
print(f"  Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"  Model dtype: {model.dtype}")
print(f"  Device: {model.device}")
print(f"  Model class: {model.__class__.__name__}")

# Check model architecture
if hasattr(model, 'config'):
    print(f"  Hidden size: {model.config.hidden_size}")
    print(f"  Num layers: {model.config.num_hidden_layers}")
    print(f"  Num heads: {model.config.num_attention_heads}")

Loading tokenizer: Qwen/Qwen2.5-0.5B-Instruct


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!



TOKENIZER DEBUG INFO
  Vocab size: 151643
  Pad token: <|endoftext|> (id=151643)
  EOS token: <|im_end|> (id=151645)
  Padding side: left
  Has chat template: True
  Chat template test: '<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|...'

LOADING MODEL: Qwen/Qwen2.5-0.5B-Instruct

MODEL DEBUG INFO
  Total parameters: 494,032,768
  Trainable parameters: 494,032,768
  Model dtype: torch.bfloat16
  Device: cuda:0
  Model class: Qwen2ForCausalLM
  Hidden size: 896
  Num layers: 24
  Num heads: 14


In [9]:
# Configure LoRA
print(f"\n{'='*60}")
print("LORA CONFIGURATION DEBUG")
print(f"{'='*60}")
print(f"  Rank (r): {LORA_R}")
print(f"  Alpha: {LORA_ALPHA}")
print(f"  Dropout: {LORA_DROPOUT}")
print(f"  Target modules: {LORA_TARGET_MODULES}")

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Apply LoRA
model = get_peft_model(model, lora_config)

print(f"\nLoRA applied successfully!")
model.print_trainable_parameters()

# Debug: Show which layers have LoRA
print(f"\nLoRA MODULES DEBUG:")
lora_layers = [name for name, _ in model.named_modules() if 'lora' in name.lower()]
print(f"  Total LoRA modules: {len(lora_layers)}")
print(f"  Sample LoRA layers: {lora_layers[:5]}")


LORA CONFIGURATION DEBUG
  Rank (r): 32
  Alpha: 64
  Dropout: 0.05
  Target modules: ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']

LoRA applied successfully!
trainable params: 17,596,416 || all params: 511,629,184 || trainable%: 3.4393

LoRA MODULES DEBUG:
  Total LoRA modules: 1512
  Sample LoRA layers: ['base_model.model.model.layers.0.self_attn.q_proj.lora_dropout', 'base_model.model.model.layers.0.self_attn.q_proj.lora_dropout.default', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.default', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B']


## 8. Train

In [ ]:
from trl import SFTTrainer, SFTConfig
from transformers import set_seed, TrainerCallback, EarlyStoppingCallback
from sklearn.metrics import accuracy_score
import re

# Set seed
set_seed(SEED)

# ============================================================
# SIMPLIFIED CALLBACK: Direct Intent Accuracy
# ============================================================

def extract_intent_direct(response, intent_tags):
    """Extract intent from direct model response (just the tag)."""
    response = response.strip().lower()
    
    # Direct match - the response should BE the intent tag
    for intent in intent_tags:
        if intent.lower() == response:
            return intent
    
    # Partial match - if the response contains an intent tag
    for intent in intent_tags:
        if intent.lower() in response:
            return intent
    
    return None

def evaluate_model_direct(model, tokenizer, eval_df, batch_size=32, num_samples=None):
    """Evaluate model with direct classification format."""
    model.eval()

    if num_samples:
        eval_df = eval_df.sample(n=min(num_samples, len(eval_df)), random_state=42).reset_index(drop=True)

    predictions = []
    true_labels = []
    raw_outputs = []

    num_batches = (len(eval_df) + batch_size - 1) // batch_size
    for i in tqdm(range(0, len(eval_df), batch_size), total=num_batches, desc="Evaluating"):
        batch_df = eval_df.iloc[i:i+batch_size]

        # Create minimal prompts
        batch_prompts = []
        for q in batch_df['question']:
            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": q}
            ]
            prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            batch_prompts.append(prompt)

        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True,
                          truncation=True, max_length=MAX_SEQ_LENGTH)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
            )

        input_len = inputs['input_ids'].shape[1]
        for j, output in enumerate(outputs):
            response = tokenizer.decode(output[input_len:], skip_special_tokens=True)
            if len(raw_outputs) < 10:
                raw_outputs.append(response[:80])
            predictions.append(extract_intent_direct(response, INTENT_TAGS))

        true_labels.extend(batch_df['tag'].tolist())

    # Show sample outputs
    print("\nSample outputs:")
    for i, resp in enumerate(raw_outputs[:10]):
        true = true_labels[i]
        pred = predictions[i]
        match = "✓" if pred == true else "✗"
        print(f"  [{i+1}] {match} '{resp}' | True: {true}")

    return predictions, true_labels

def classify_intent(query, model, tokenizer):
    """Classify intent for a single Bengali query."""
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": query}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LENGTH)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
        )

    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    intent = extract_intent_direct(response, INTENT_TAGS)

    return intent, response

class IntentAccuracyCallback(TrainerCallback):
    """Callback to compute intent accuracy during training."""

    def __init__(self, eval_df, tokenizer, intent_tags, max_seq_length, 
                 sample_size=500, batch_size=32, patience=3):
        self.eval_sample = eval_df.sample(n=min(sample_size, len(eval_df)), random_state=42).reset_index(drop=True)
        self.tokenizer = tokenizer
        self.intent_tags = intent_tags
        self.max_seq_length = max_seq_length
        self.batch_size = batch_size
        self.patience = patience
        self.best_accuracy = 0.0
        self.no_improve_count = 0
        self.history = []
        self.train_losses = []
        print(f"[CALLBACK INIT] Eval samples: {len(self.eval_sample)}, batch_size={batch_size}")

    def on_train_begin(self, args, state, control, **kwargs):
        print(f"\n{'='*60}")
        print("[DEBUG] TRAINING STARTED")
        print(f"{'='*60}")
        print(f"  Max steps: {state.max_steps}")

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and 'loss' in logs:
            self.train_losses.append(logs['loss'])
            if state.global_step % 100 == 0:
                avg = sum(self.train_losses[-10:]) / min(10, len(self.train_losses))
                print(f"  [Step {state.global_step}] loss={logs['loss']:.4f} | avg_10={avg:.4f}")

    def _compute_accuracy_batched(self, model):
        """Compute intent accuracy on sample using batched inference."""
        torch.cuda.empty_cache()
        model.eval()
        predictions = []
        true_labels = self.eval_sample['tag'].tolist()
        debug_responses = []

        for i in range(0, len(self.eval_sample), self.batch_size):
            batch_df = self.eval_sample.iloc[i:i+self.batch_size]

            # Create minimal prompts
            batch_prompts = []
            for q in batch_df['question']:
                messages = [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": q}
                ]
                prompt = self.tokenizer.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True
                )
                batch_prompts.append(prompt)

            inputs = self.tokenizer(batch_prompts, return_tensors="pt", padding=True,
                                   truncation=True, max_length=self.max_seq_length)
            inputs = {k: v.to(model.device) for k, v in inputs.items()}

            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=50,  # Short - just the tag
                    do_sample=False,
                    pad_token_id=self.tokenizer.pad_token_id,
                )

            input_len = inputs['input_ids'].shape[1]
            for idx, output in enumerate(outputs):
                response = self.tokenizer.decode(output[input_len:], skip_special_tokens=True)
                if len(debug_responses) < 5:
                    debug_responses.append(response[:100])
                predictions.append(extract_intent_direct(response, self.intent_tags))

        # Debug output
        print(f"\n    [DEBUG] Sample outputs (first 5):")
        for i, resp in enumerate(debug_responses[:5]):
            true = true_labels[i]
            pred = predictions[i]
            match = "✓" if pred == true else "✗"
            print(f"      {match} Raw: '{resp}' | True: {true} | Pred: {pred}")

        # Compute accuracy
        valid_pairs = [(p, t) for p, t in zip(predictions, true_labels) if p is not None]
        num_none = sum(1 for p in predictions if p is None)
        print(f"    [DEBUG] Valid: {len(valid_pairs)}/{len(predictions)} ({num_none} None)")

        if len(valid_pairs) == 0:
            return 0.0
        valid_preds, valid_true = zip(*valid_pairs)
        return accuracy_score(valid_true, valid_preds)

    def on_evaluate(self, args, state, control, model, **kwargs):
        accuracy = self._compute_accuracy_batched(model)
        self.history.append({'step': state.global_step, 'intent_accuracy': accuracy})

        print(f"\n>>> Intent Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

        if accuracy > self.best_accuracy + 0.001:
            self.best_accuracy = accuracy
            self.no_improve_count = 0
            print(f"    [NEW BEST] Best accuracy: {self.best_accuracy:.4f}")
        else:
            self.no_improve_count += 1
            print(f"    No improvement for {self.no_improve_count}/{self.patience} evals")

        if self.no_improve_count >= self.patience:
            print(f"\n*** EARLY STOPPING: No improvement for {self.patience} evals ***")
            control.should_training_stop = True

        return control

# ============================================================
# TRAINING CONFIGURATION - Optimized for L4 24GB
# ============================================================

training_args = SFTConfig(
    output_dir=OUTPUT_DIR,

    # Training schedule
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,

    # Optimizer
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    warmup_ratio=WARMUP_RATIO,
    lr_scheduler_type="cosine",
    optim="adamw_8bit",

    # Mixed precision
    bf16=True,

    # Packing for efficiency
    packing=True,
    max_length=MAX_SEQ_LENGTH,

    # Logging & saving
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,

    # Other
    seed=SEED,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Create callbacks
intent_callback = IntentAccuracyCallback(
    eval_df=eval_df,
    tokenizer=tokenizer,
    intent_tags=INTENT_TAGS,
    max_seq_length=MAX_SEQ_LENGTH,
    sample_size=500,
    batch_size=EVAL_BATCH_SIZE,
    patience=EARLY_STOPPING_PATIENCE,
)

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=EARLY_STOPPING_PATIENCE,
    early_stopping_threshold=0.001,
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=training_args,
    callbacks=[early_stopping_callback, intent_callback],
)

# ============================================================
# DEBUG: Training setup info
# ============================================================
print(f"\n{'='*60}")
print("TRAINING SETUP DEBUG INFO")
print(f"{'='*60}")
print(f"  Model: {MODEL_NAME}")
print(f"  Max sequence length: {MAX_SEQ_LENGTH}")
print(f"  Packing: ENABLED")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Gradient accumulation: {GRAD_ACCUM_STEPS}")
print(f"  Effective batch size: {BATCH_SIZE * GRAD_ACCUM_STEPS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Eval every: {training_args.eval_steps} steps")
print(f"  Early stopping patience: {EARLY_STOPPING_PATIENCE}")

# Estimate VRAM
print(f"\n  Estimated VRAM usage: ~12-16GB (L4 has 24GB)")

# Quick generation test
print(f"\nQUICK GENERATION TEST (before training):")
test_messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "একাউন্ট লক হয়ে গেছে"}
]
test_prompt = tokenizer.apply_chat_template(test_messages, tokenize=False, add_generation_prompt=True)
test_inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    test_out = model.generate(**test_inputs, max_new_tokens=30, do_sample=False, pad_token_id=tokenizer.pad_token_id)
test_response = tokenizer.decode(test_out[0][test_inputs['input_ids'].shape[1]:], skip_special_tokens=True)
print(f"  Query: একাউন্ট লক হয়ে গেছে")
print(f"  Output: '{test_response}'")
print(f"\nTrainer ready!")

In [ ]:
# Train!
print("Starting training...")
print(f"Dataset: ~78k Bengali NID queries, 407 intents")
print(f"Format: DIRECT CLASSIFICATION (no CoT, no few-shot)")
print(f"Epochs: {NUM_EPOCHS} (with early stopping)")
print(f"Packing: ENABLED for efficiency")
print("")
print("Expected behavior:")
print("  - Model outputs just the intent tag (e.g., 'account_locked')")
print("  - No reasoning, no examples, just direct classification")
print("-" * 50)

trainer.train(resume_from_checkpoint=RESUME_FROM_CHECKPOINT)

# ============================================================
# POST-TRAINING DEBUG INFO
# ============================================================
print(f"\n{'='*60}")
print("TRAINING COMPLETE - DEBUG SUMMARY")
print(f"{'='*60}")
print(f"  Best intent accuracy: {intent_callback.best_accuracy:.4f} ({intent_callback.best_accuracy*100:.2f}%)")
print(f"  Total evaluations: {len(intent_callback.history)}")

# Show accuracy progression
print(f"\nACCURACY PROGRESSION:")
for h in intent_callback.history:
    print(f"  Step {h['step']}: {h['intent_accuracy']:.4f} ({h['intent_accuracy']*100:.1f}%)")

# Post-training generation test
print(f"\nPOST-TRAINING GENERATION TEST:")
test_queries = [
    "একাউন্ট লক হয়ে গেছে কি করব?",
    "কার্ড হারিয়ে গেছে",
    "নাম সংশোধন করতে চাই"
]
for q in test_queries:
    test_messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": q}
    ]
    test_prompt = tokenizer.apply_chat_template(test_messages, tokenize=False, add_generation_prompt=True)
    test_inputs = tokenizer(test_prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LENGTH).to(model.device)
    with torch.no_grad():
        test_out = model.generate(**test_inputs, max_new_tokens=50, do_sample=False, pad_token_id=tokenizer.pad_token_id)
    test_response = tokenizer.decode(test_out[0][test_inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    print(f"  Q: {q}")
    print(f"  A: {test_response}")
# Save SFT model + mappings before DPO
print(f"\nSaving SFT model to {OUTPUT_DIR}...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

import json
with open(f"{OUTPUT_DIR}/intent_mappings.json", "w", encoding="utf-8") as f:
    json.dump({"id2intent": ID2INTENT, "intent2id": INTENT2ID}, f, ensure_ascii=False, indent=2)

with open(f"{OUTPUT_DIR}/training_metadata.json", "w", encoding="utf-8") as f:
    json.dump({
        "stage": "sft",
        "base_model": MODEL_NAME,
        "run_name": RUN_NAME,
        "output_dir": OUTPUT_DIR,
        "dpo_output_dir": DPO_OUTPUT_DIR,
    }, f, ensure_ascii=False, indent=2)

print("SFT model saved successfully!")
print(f"Files in {OUTPUT_DIR}:")
!ls -la {OUTPUT_DIR}


## 8.1 DPO Training (Preference Optimization)

After SFT, we apply DPO to improve accuracy on hard/ambiguous intents.
This trains the model to prefer correct intents over wrong predictions.

In [ ]:
# ============================================================
# STEP 1: Build DPO Preference Dataset from Hard Cases
# ============================================================

from collections import defaultdict
from tqdm import tqdm

def create_dpo_dataset_from_errors(model, tokenizer, eval_df, intent_tags,
                                    batch_size=32, num_samples=5000):
    """
    Run inference on eval set, collect wrong predictions as preference pairs.
    Format: prompt -> (chosen=correct_tag, rejected=wrong_prediction)
    """
    model.eval()
    preference_pairs = []
    
    sample_df = eval_df.sample(n=min(num_samples, len(eval_df)), random_state=42).reset_index(drop=True)
    
    print(f"Collecting model errors from {len(sample_df)} samples...")
    for i in tqdm(range(0, len(sample_df), batch_size), desc="Collecting errors"):
        batch_df = sample_df.iloc[i:i+batch_size]
        
        # Create prompts
        batch_prompts = []
        for q in batch_df['question']:
            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": q}
            ]
            prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            batch_prompts.append(prompt)
        
        # Generate predictions
        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True,
                          truncation=True, max_length=MAX_SEQ_LENGTH)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=50, do_sample=False,
                                    pad_token_id=tokenizer.pad_token_id)
        
        input_len = inputs['input_ids'].shape[1]
        for idx, (output, (_, row)) in enumerate(zip(outputs, batch_df.iterrows())):
            response = tokenizer.decode(output[input_len:], skip_special_tokens=True).strip()
            true_tag = row['tag']
            pred_tag = extract_intent_direct(response, intent_tags)
            
            # Only collect WRONG predictions as preference pairs
            if pred_tag is not None and pred_tag != true_tag:
                preference_pairs.append({
                    "prompt": batch_prompts[idx],
                    "chosen": true_tag,      # Correct intent
                    "rejected": pred_tag,    # Model's wrong prediction
                })
    
    return preference_pairs

def add_synthetic_negatives(train_df, tokenizer, num_samples=3000):
    """Create pairs with similar-looking tags as negatives."""
    pairs = []
    tags_by_prefix = defaultdict(list)
    
    # Group tags by prefix (e.g., address_change_*)
    for tag in train_df['tag'].unique():
        prefix = tag.split('_')[0]
        tags_by_prefix[prefix].append(tag)
    
    print(f"Creating synthetic hard negatives from {num_samples} samples...")
    sample_df = train_df.sample(n=min(num_samples, len(train_df)), random_state=42)
    
    for _, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="Synthetic pairs"):
        true_tag = row['tag']
        prefix = true_tag.split('_')[0]
        
        # Pick a similar tag (same prefix) as hard negative
        similar_tags = [t for t in tags_by_prefix[prefix] if t != true_tag]
        if similar_tags:
            wrong_tag = random.choice(similar_tags)
            
            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": row['question']}
            ]
            prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            
            pairs.append({
                "prompt": prompt,
                "chosen": true_tag,
                "rejected": wrong_tag,
            })
    
    return pairs

# Collect error-based preference pairs
print(f"\n{'='*60}")
print("BUILDING DPO PREFERENCE DATASET")
print(f"{'='*60}")

error_pairs = create_dpo_dataset_from_errors(
    model, tokenizer, eval_df, INTENT_TAGS, 
    batch_size=EVAL_BATCH_SIZE, num_samples=5000
)
print(f"  Error pairs collected: {len(error_pairs)}")

# Add synthetic hard negatives
synthetic_pairs = add_synthetic_negatives(train_df, tokenizer, num_samples=3000)
print(f"  Synthetic pairs created: {len(synthetic_pairs)}")

# Combine all pairs
all_pairs = error_pairs + synthetic_pairs
random.shuffle(all_pairs)
print(f"  Total DPO pairs: {len(all_pairs)}")

# Show sample pairs
print(f"\nSample preference pairs:")
for i, pair in enumerate(all_pairs[:3]):
    print(f"  [{i+1}] Chosen: {pair['chosen']} | Rejected: {pair['rejected']}")

In [ ]:
# ============================================================
# STEP 2: DPO Training
# ============================================================

from trl import DPOTrainer, DPOConfig
from datasets import Dataset

# Create HuggingFace Dataset from preference pairs
dpo_dataset = Dataset.from_list(all_pairs)

print(f"\n{'='*60}")
print("DPO TRAINING CONFIGURATION")
print(f"{'='*60}")

# DPO Config - optimized for L4 24GB
dpo_config = DPOConfig(
    output_dir=DPO_OUTPUT_DIR,
    
    # Training schedule
    num_train_epochs=2,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    
    # DPO-specific
    beta=0.1,  # Preference strength (0.05-0.2 range)
    loss_type="sigmoid",  # Best for classification
    
    # Sequence lengths
    max_prompt_length=200,
    max_length=256,
    
    # Optimizer
    learning_rate=5e-7,  # Low LR for preference fine-tuning
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="adamw_8bit",
    
    # Memory
    bf16=True,
    gradient_checkpointing=True,
    
    # Logging
    logging_steps=20,
    eval_strategy="no",  # Skip eval during DPO
    save_strategy="epoch",
    save_total_limit=2,
    
    seed=SEED,
    report_to="none",
)

print(f"  Preference pairs: {len(dpo_dataset)}")
print(f"  Beta: {dpo_config.beta}")
print(f"  Learning rate: {dpo_config.learning_rate}")
print(f"  Batch size: {dpo_config.per_device_train_batch_size} x {dpo_config.gradient_accumulation_steps}")
print(f"  Epochs: {dpo_config.num_train_epochs}")

# Initialize DPO Trainer
dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,  # Auto-creates frozen copy
    args=dpo_config,
    train_dataset=dpo_dataset,
    processing_class=tokenizer,
)

print(f"\n{'='*60}")
print("STARTING DPO TRAINING")
print(f"{'='*60}")
print("This will teach the model to prefer correct intents over wrong ones...")

dpo_trainer.train(resume_from_checkpoint=DPO_RESUME_FROM_CHECKPOINT)

print(f"\n{'='*60}")
print("DPO TRAINING COMPLETE")
print(f"{'='*60}")

In [ ]:
# ============================================================
# STEP 3: Evaluate DPO-improved Model
# ============================================================

print(f"\n{'='*60}")
print("POST-DPO EVALUATION")
print(f"{'='*60}")

# Re-run evaluation on the same samples
predictions_dpo, true_labels_dpo = evaluate_model_direct(
    model, tokenizer, eval_df, batch_size=EVAL_BATCH_SIZE, num_samples=2000
)

# Calculate metrics
valid_mask_dpo = [p is not None for p in predictions_dpo]
valid_preds_dpo = [INTENT2ID.get(p, -1) for p in predictions_dpo]
valid_true_dpo = [INTENT2ID.get(t, -1) for t in true_labels_dpo]

filtered_preds_dpo = [p for p, m in zip(valid_preds_dpo, valid_mask_dpo) if m and p != -1]
filtered_true_dpo = [t for t, m, p in zip(valid_true_dpo, valid_mask_dpo, valid_preds_dpo) if m and p != -1]

if len(filtered_preds_dpo) > 0:
    accuracy_dpo = accuracy_score(filtered_true_dpo, filtered_preds_dpo)
else:
    accuracy_dpo = 0.0

null_preds_dpo = sum(1 for p in predictions_dpo if p is None)

# Compare results
print(f"\n{'='*60}")
print("RESULTS COMPARISON: Before vs After DPO")
print(f"{'='*60}")
print(f"  Before DPO: ~70.0% accuracy, ~0.4% None predictions")
print(f"  After DPO:  {accuracy_dpo*100:.1f}% accuracy, {100*null_preds_dpo/len(predictions_dpo):.1f}% None")
print(f"")
if accuracy_dpo > 0.70:
    improvement = (accuracy_dpo - 0.70) * 100
    print(f"  Improvement: +{improvement:.1f}%")
else:
    print(f"  Note: Run more epochs or adjust beta if accuracy didn't improve")

# Show sample outputs after DPO
print(f"\nPOST-DPO TEST QUERIES:")
test_queries_dpo = [
    "একাউন্ট লক হয়ে গেছে কি করব?",
    "কার্ড হারিয়ে গেছে",
    "নাম সংশোধন করতে চাই",
    "ঠিকানা পরিবর্তন করতে চাই",
]
for q in test_queries_dpo:
    intent, raw = classify_intent(q, model, tokenizer)
    print(f"  Q: {q}")
    print(f"  A: {raw} -> {intent}")

## 9. Save DPO Model


In [ ]:
# Save DPO model to Google Drive
if 'dpo_trainer' in globals():
    print(f"Saving DPO model to {DPO_OUTPUT_DIR}...")
    dpo_trainer.save_model(DPO_OUTPUT_DIR)
    tokenizer.save_pretrained(DPO_OUTPUT_DIR)

    # Also save intent mappings + metadata
    import json
    with open(f"{DPO_OUTPUT_DIR}/intent_mappings.json", "w", encoding="utf-8") as f:
        json.dump({"id2intent": ID2INTENT, "intent2id": INTENT2ID}, f, ensure_ascii=False, indent=2)

    with open(f"{DPO_OUTPUT_DIR}/training_metadata.json", "w", encoding="utf-8") as f:
        json.dump({
            "stage": "dpo",
            "base_model": MODEL_NAME,
            "run_name": RUN_NAME,
            "output_dir": OUTPUT_DIR,
            "dpo_output_dir": DPO_OUTPUT_DIR,
        }, f, ensure_ascii=False, indent=2)

    print("DPO model saved successfully!")
    print(f"Files in {DPO_OUTPUT_DIR}:")
    !ls -la {DPO_OUTPUT_DIR}
else:
    print("DPO trainer not found. Skipping DPO save.")


## 10. Evaluate

In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from collections import Counter

# Evaluate
print(f"\n{'='*60}")
print("EVALUATION")
print(f"{'='*60}")
print(f"  Batch size: {EVAL_BATCH_SIZE}")
print(f"  Num samples: 2000")
print(f"\nStarting evaluation...")

predictions, true_labels = evaluate_model_direct(
    model, tokenizer, eval_df, batch_size=EVAL_BATCH_SIZE, num_samples=2000
)

# Debug: Prediction distribution
print(f"\nPREDICTION DEBUG:")
null_preds = sum(1 for p in predictions if p is None)
print(f"  Null predictions: {null_preds}/{len(predictions)} ({100*null_preds/len(predictions):.1f}%)")
pred_counts = Counter([p for p in predictions if p])
top_preds = pred_counts.most_common(10)
print(f"  Top 10 predicted tags: {top_preds}")


In [ ]:
# Compute metrics
valid_mask = [p is not None for p in predictions]
valid_preds = [INTENT2ID.get(p, -1) for p in predictions]
valid_true = [INTENT2ID.get(t, -1) for t in true_labels]

# Filter valid
filtered_preds = [p for p, m in zip(valid_preds, valid_mask) if m and p != -1]
filtered_true = [t for t, m, p in zip(valid_true, valid_mask, valid_preds) if m and p != -1]

# Calculate metrics
if len(filtered_preds) > 0:
    accuracy = accuracy_score(filtered_true, filtered_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        filtered_true, filtered_preds, average="weighted", zero_division=0
    )
else:
    accuracy = precision = recall = f1 = 0.0

print("=" * 50)
print("EVALUATION RESULTS")
print("=" * 50)
print(f"Total samples: {len(predictions)}")
print(f"Valid predictions: {sum(valid_mask)} ({100*sum(valid_mask)/len(predictions):.1f}%)")
print(f"")
print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print("=" * 50)

# Show top confusions
print("\nTop 10 Confusions (True -> Predicted):")
confusions = [(t, p) for p, t in zip(predictions, true_labels) if p != t and p is not None]
for (true, pred), count in Counter(confusions).most_common(10):
    print(f"  {true} -> {pred}: {count}")

## 11. Interactive Inference

In [ ]:
# Get answer for intent from tag_answer_df
def get_answer(intent):
    """Get Bengali answer for an intent."""
    row = tag_answer_df[tag_answer_df['tag'] == intent]
    if len(row) > 0:
        return row.iloc[0]['answer']
    return "উত্তর পাওয়া যায়নি।"

# Test with sample Bengali queries
test_queries = [
    "আমার এনআইডি একাউন্ট লক হয়ে গেছে, কিভাবে আনলক করবো?",
    "কার্ড হারিয়ে গেলে কি করতে হবে?",
    "জাতীয় পরিচয়পত্রে নাম সংশোধন করতে চাই",
    "ভোটার আইডি কার্ডের ঠিকানা পরিবর্তন করতে কি কি লাগবে?",
    "স্মার্ট কার্ড কবে পাবো?",
    "আমার জন্ম তারিখ ভুল আছে",
    "NID নম্বর ভুলে গেছি",
]

print("Testing with Bengali queries (Direct Classification):")
print("=" * 70)
for query in test_queries:
    intent, raw_output = classify_intent(query, model, tokenizer)
    answer = get_answer(intent) if intent else "Intent not recognized"
    print(f"Query: {query}")
    print(f"Raw output: '{raw_output}'")
    print(f"Intent: {intent}")
    print(f"Answer: {answer[:100]}..." if len(answer) > 100 else f"Answer: {answer}")
    print("-" * 70)


## 12. Push to HuggingFace Hub

In [ ]:
from huggingface_hub import login

if not HF_TOKEN:
    raise ValueError("Set HF_TOKEN in the top cell before login.")

login(token=HF_TOKEN)


In [ ]:
from datetime import datetime

if not HF_USERNAME:
    raise ValueError("Set HF_USERNAME in the top cell.")

if not HF_REPO_SUFFIX:
    HF_REPO_SUFFIX = f"{RUN_NAME}-{MODEL_NAME.split('/')[-1].lower()}"

if not HF_RUN_TAG:
    HF_RUN_TAG = datetime.utcnow().strftime("%Y%m%d-%H%M%S")

HF_REPO_NAME = f"{HF_USERNAME}/{HF_REPO_SUFFIX}-{HF_RUN_TAG}"

print(f"Pushing model to HuggingFace Hub: {HF_REPO_NAME}")

model.push_to_hub(HF_REPO_NAME)
tokenizer.push_to_hub(HF_REPO_NAME)

print(f"\nModel uploaded successfully!")
print(f"View at: https://huggingface.co/{HF_REPO_NAME}")


## Done!

Your Bengali NID intent classification model has been trained with:

### Training Pipeline:
1. **SFT (Supervised Fine-Tuning)** - Direct classification format (~70% accuracy)
2. **DPO (Direct Preference Optimization)** - Improves hard/ambiguous intents (+5-10%)

### Key Features:
- **Direct Classification** - No CoT reasoning, no few-shot examples
- **Minimal Format** - System prompt + query → tag only
- **Packing Enabled** - More efficient GPU utilization
- **Optimized for L4** - ~16GB VRAM usage
- **DPO on errors** - Trains on actual model mistakes + hard negatives

### Model Locations:
- SFT output: `/content/drive/MyDrive/models/smollm2-bengali-nid-intent`
- DPO output: `/content/drive/MyDrive/models/smollm2-bengali-nid-intent-dpo`
- HuggingFace Hub: `HF_REPO_NAME` printed in the upload cell

### To load the model later (HF or local checkpoint):
```python
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import glob

# Option A: HuggingFace Hub (use the printed HF_REPO_NAME)
HF_REPO_NAME = "your-username/bn-nid-intent-qwen2.5-0.5b-YYYYMMDD-HHMMSS"
base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
model = PeftModel.from_pretrained(base_model, HF_REPO_NAME)
tokenizer = AutoTokenizer.from_pretrained(HF_REPO_NAME)

# Option B: Local checkpoint
# ckpts = sorted(glob.glob("/content/drive/MyDrive/models/smollm2-bengali-nid-intent/checkpoint-*"))
# checkpoint = ckpts[-1] if ckpts else "/content/drive/MyDrive/models/smollm2-bengali-nid-intent"
# model = PeftModel.from_pretrained(base_model, checkpoint)
# tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/models/smollm2-bengali-nid-intent")

# Inference
SYSTEM_PROMPT = "You are an intent classifier for Bengali NID customer service. Output only the intent tag, nothing else."
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "একাউন্ট লক হয়ে গেছে"}
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=False)
intent = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
print(intent)  # Should output: account_locked
```

### Expected Results:
| Stage | Accuracy | None % |
|-------|----------|--------|
| After SFT | ~70% | ~0.4% |
| After DPO | ~75-80% | ~0% |